In [9]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [10]:
# Load API Credentials
with open('/Users/kass/secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [11]:
# set our API call parameters 
LOCATION = 'Boston, MA'
TERM = 'Vegan'

In [12]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_Bos_Vegan.json"
JSON_FILE

'Data/results_in_progress_Bos_Vegan.json'

In [15]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [16]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [17]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [18]:
## How many results total?
total_results = results['total']
total_results

2200

In [19]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [20]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

110

In [21]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [22]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/110 [00:00<?, ?it/s]

In [23]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,hdBsXpL9uAllcb2QsqzK7g,redwhite-boneless-ramen-boston-3,REDWHITE BONELESS RAMEN,https://s3-media1.fl.yelpcdn.com/bphoto/nr-GE1...,False,https://www.yelp.com/biz/redwhite-boneless-ram...,409,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 42.348815, 'longitude': -71.084731}","[delivery, pickup]",$$,"{'address1': '294 Newbury St', 'address2': Non...",18572770609,(857) 277-0609,1111.128590
1,mnFhrH1Fi5GoSuV-5gVpnA,krasi-boston-2,Krasi,https://s3-media2.fl.yelpcdn.com/bphoto/OCJ5vI...,False,https://www.yelp.com/biz/krasi-boston-2?adjust...,376,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.5,"{'latitude': 42.34872306019752, 'longitude': -...",[delivery],$$,"{'address1': '48 Gloucester St', 'address2': '...",16175360230,(617) 536-0230,1056.307606
2,kP1b-7BO_VhWk_0tvuA_tw,carmelinas-boston-2,Carmelina's,https://s3-media2.fl.yelpcdn.com/bphoto/rxZBwI...,False,https://www.yelp.com/biz/carmelinas-boston-2?a...,3328,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 42.36388, 'longitude': -71.05415}","[restaurant_reservation, delivery]",$$,"{'address1': '307 Hanover St', 'address2': '',...",16177420020,(617) 742-0020,2272.117192
3,lsqYQYGfpe25wUZk9Wybwg,the-elephant-walk-boston-2,The Elephant Walk,https://s3-media2.fl.yelpcdn.com/bphoto/jnWAud...,False,https://www.yelp.com/biz/the-elephant-walk-bos...,278,"[{'alias': 'cambodian', 'title': 'Cambodian'},...",4.5,"{'latitude': 42.34116, 'longitude': -71.070578}","[delivery, pickup]",$$,"{'address1': '1415 Washington St', 'address2':...",16172471500,(617) 247-1500,744.978862
4,Q8g68ZDAt93h_n_wTbwshA,shōjō-boston-boston-2,Shōjō-Boston,https://s3-media1.fl.yelpcdn.com/bphoto/CSSmYu...,False,https://www.yelp.com/biz/sh%C5%8Dj%C5%8D-bosto...,989,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4.0,"{'latitude': 42.35113, 'longitude': -71.06077}","[delivery, pickup]",$$,"{'address1': '9A Tyler St', 'address2': None, ...",16174237888,(617) 423-7888,932.457881


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
15,nIS4O-TwLYkA_P1Cu0SfhA,yvonnes-boston,Yvonne's,https://s3-media3.fl.yelpcdn.com/bphoto/Ob4EQb...,False,https://www.yelp.com/biz/yvonnes-boston?adjust...,1363,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 42.355517, 'longitude': -71.0614361}",[delivery],$$,"{'address1': '2 Winter Pl', 'address2': None, ...",16172670047,(617) 267-0047,1173.830091
16,lPAzwB4vPBqZmaFNH90kiQ,cafe-bonjour-boston,Cafe Bonjour,https://s3-media2.fl.yelpcdn.com/bphoto/CddDi5...,False,https://www.yelp.com/biz/cafe-bonjour-boston?a...,725,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.5,"{'latitude': 42.354838, 'longitude': -71.061816}","[delivery, pickup]",$$,"{'address1': '55 Temple Pl', 'address2': '', '...",16177790062,(617) 779-0062,1098.043853
17,XnQYoHy23BULUooiqHe42Q,lulu-green-boston,Lulu Green,https://s3-media3.fl.yelpcdn.com/bphoto/jUVAsz...,False,https://www.yelp.com/biz/lulu-green-boston?adj...,90,"[{'alias': 'vegan', 'title': 'Vegan'}, {'alias...",4.5,"{'latitude': 42.339588, 'longitude': -71.051715}","[delivery, pickup]",$$,"{'address1': '246 W Broadway', 'address2': Non...",16174204070,(617) 420-4070,1850.819001
18,YGoNygBanbHP7bpkpnrv7g,life-alive-organic-cafe-back-bay-boston,Life Alive Organic Cafe Back Bay,https://s3-media2.fl.yelpcdn.com/bphoto/j39-pY...,False,https://www.yelp.com/biz/life-alive-organic-ca...,253,"[{'alias': 'juicebars', 'title': 'Juice Bars &...",4.0,"{'latitude': 42.351508, 'longitude': -71.072847}","[delivery, pickup]",$$,"{'address1': '431 Boylston St', 'address2': No...",18572398239,(857) 239-8239,433.438408
19,pB_mOAgN7JuOdBp0HDA4Hg,plnt-burger-boston,PLNT Burger,https://s3-media2.fl.yelpcdn.com/bphoto/bYO2f5...,False,https://www.yelp.com/biz/plnt-burger-boston?ad...,15,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"{'latitude': 42.34546037805935, 'longitude': -...","[delivery, pickup]",NaN,"{'address1': '348 Harrison Ave', 'address2': N...",16178822989,(617) 882-2989,743.951912


In [24]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [25]:


## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [26]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_pizza.csv.gz', compression='gzip',index=False)